# Running DALI

In [ ]:
# Import predicted structures, present in the strucs directory, to dali database format
vpSAT/bin/dali_format_inputs.sh \
-d strucs \
-o db \
-s ligt_key.txt 

# Run DALI 
dali_matrix.sh \
-q db \
-o dali_results/ligTs \
-n 5

# Parse the DALI output files
sat.py aln_parse_dali_matrix \
-k ligt_key.txt \
-t dali_results/ligTs.newick_unrooted \
-T dali_results/ligTs.newick_unrooted.PARSED \
-m dali_results/ligTs.sim.txt \
-M dali_results/ligTs.sim.PARSED.txt

# Plotting the Z-score similarity matrix

In [ ]:
### The following is R code

library(tidyverse)
library(RColorBrewer)

m <- read_tsv("dali_results/ligTs.sim.PARSED.txt", col_names="")

order = m$...1

colnames(m) <- c("name" ,order)


plt <- m %>%
  pivot_longer(
    -name,
    names_to = "name2",
    values_to = "value"
  ) %>%
  
  mutate(name = factor(name, ordered=T, levels=order)) %>%
  mutate(name2 = factor(name2, ordered=T, levels=rev(order))) %>%
  
  filter(name != name2) %>%

  ggplot() +
  
  geom_tile(
    aes(x=name, y=name2, fill=value)
  ) +
  
  theme_minimal() + 
  theme(axis.text.x = element_blank(),
        axis.ticks = element_blank(),
        legend.position = "top",
        axis.title = element_blank(),
        panel.grid = element_blank()) +
  labs(fill = "Z") +
  scale_y_discrete(position = "right", expand = c(0,0)) +
  scale_x_discrete(expand = c(0,0)) +
  scale_fill_distiller(palette = "YlGnBu", trans="reverse")
  

ggsave("dali_matrix.svg", plt, height=10, width=12, units="in")